In [2]:
pip install selenium

  Using cached selenium-4.19.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached trio-0.25.0-py3-none-any.whl.metadata (8.7 kB)
  Using cached trio_websocket-0.11.1-py3-none-any.whl.metadata (4.7 kB)
  Using cached attrs-23.2.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
Using cached selenium-4.19.0-py3-none-any.whl (10.5 MB)
Using cached trio-0.25.0-py3-none-any.whl (467 kB)
Using cached trio_websocket-0.11.1-py3-none-any.whl (17 kB)
Using cached attrs-23.2.0-py3-none-any.whl (60 kB)
Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)
Using cached outcome-1.3.0.post0-py2.py3-none-any.whl (10 kB)
Using cached h11-0.14.0-py3-none-any.whl (58 kB)
  Attempting uninstall: attrs
    Found existing installation: attrs 23.1.0
    Uninstalling attrs-23.1.0:
      Successfully uninstalled attrs-23.1.0
Note: 

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

# Prompt the user to input the keyword
keyword = input("Enter the job keyword: ")

# Initialize Safari WebDriver
driver = webdriver.Safari()

# Website URL
url = "https://www.indeed.com"

# Open the URL
driver.get(url)

# Wait for the input box for "What" to be clickable
input_what = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="text-input-what"]')))

# Click on the input box for "What"
input_what.click()

# Clear the existing text in "What" box if any
input_what.clear()

# Type "Data analyst" into the input box for "What"
input_what.send_keys(keyword)

# Find and click the search button
search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="jobsearch"]/div/div[2]/button')))
search_button.click()

# Initialize empty lists to store job positions, company names, and locations
job_positions = []
company_names = []
locations = []
job_state_dates = []
requirements = []

try:
    while True:
        # Wait for the search results to load
        WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'jcs-JobTitle')))
        
        # Extract job positions, company names, and locations from the search results
        job_elements = driver.find_elements(By.CLASS_NAME, 'jcs-JobTitle')
        company_elements = driver.find_elements(By.XPATH, '//span[@data-testid="company-name" and contains(@class, "css-") and contains(@class, "eu4oa1w0")]')
        location_elements = driver.find_elements(By.XPATH, '//div[@data-testid="text-location"]')
        state_date_elements = driver.find_elements(By.XPATH, '//span[@data-testid="myJobsStateDate"]')
        requirement_elements = driver.find_elements(By.XPATH, '//div[@class="heading6 tapItem-gutter css-193h767 eu4oa1w0"]')

        
        for i in range(len(job_elements)):
            job_position = job_elements[i].text.split('(')[0].strip()  # Extract job position
            job_positions.append(job_position)
            
            company_name = company_elements[i].text if i < len(company_elements) else ""  # Extract company name
            company_names.append(company_name)
            
            location = location_elements[i].text if i < len(location_elements) else ""  # Extract location
            locations.append(location)

            job_state_date = state_date_elements[i].text.strip() if i < len(state_date_elements) else ""  # Extract job state date
            job_state_dates.append(job_state_date)
            
            requirement = requirement_elements[i].text.strip() if i < len(requirement_elements) else ""  # Extract requirement
            requirements.append(requirement)
        
        
        # Find the "Next" page button and click it if available
        try:
            next_button = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="jobsearch-JapanPage"]/div/div[5]/div/div[1]/nav/ul/li[6]/a')))
            if "disabled" in next_button.get_attribute("class"):
                break  # Exit the loop if the "Next" button is disabled

            next_button.click()
        except:
            print("Next page button not found or disabled. Exiting loop.")
            break
except:
    print("An error occurred while waiting for search results. Quitting...")
finally:
    # Quit the driver
    driver.quit()

# Ensure all lists have the same length by padding shorter lists with empty strings
max_length = max(len(job_positions), len(company_names), len(locations), len(job_state_dates), len(requirements))
job_positions += [''] * (max_length - len(job_positions))
company_names += [''] * (max_length - len(company_names))
locations += [''] * (max_length - len(locations))
job_state_dates += [''] * (max_length - len(job_state_dates))
requirements += [''] * (max_length - len(requirements))

# Creating a DataFrame from the lists of job positions, company names, and locations
df = pd.DataFrame({
    'Job Position': job_positions,
    'Company Name': company_names,
    'Location': locations,  # Adding the Location column
    'Job State Date': job_state_dates,
    'Requirement': requirements  # Adding the Requirement column
})

# Printing the DataFrame
print(df)

# Save DataFrame to Excel
excel_filename = keyword + "_results.xlsx"
df.to_excel(excel_filename, index=False)
print("Data saved to", excel_filename)

Enter the job keyword:  data analyst


Next page button not found or disabled. Exiting loop.
                                          Job Position  \
0                                Treasury Data Analyst   
1    Junior EMDI/CEDR Implementation & Quality Data...   
2                               Marketing Data Analyst   
3                                     Business Analyst   
4                Analyst, Revenue Operations Reporting   
..                                                 ...   
224                         Investor Reporting Analyst   
225  Senior Lead Data Management Analyst - Data Def...   
226                 Pricing/Profit Improvement Analyst   
227                               ERP Business Analyst   
228                     Lead Senior Accounting Analyst   

                                 Company Name  \
0                                    Comerica   
1    American College of Emergency Physicians   
2                                Smartfox LLC   
3                                  LA Fitness   
4   

In [6]:
df

,Job Position,Company Name,Location,Job State Date,Requirement
0,Treasury Data Analyst,Comerica,"Hybrid work in Frisco, TX 75034",PostedPosted 7 days ago,Provide data tracking and analytics in support...
1,Junior EMDI/CEDR Implementation & Quality Data...,American College of Emergency Physicians,"Hybrid work in Irving, TX 75063",EmployerActive 3 days ago,Experience handling a full range of delegated ...
2,Marketing Data Analyst,Smartfox LLC,"Hybrid work in Plano, TX",EmployerActive 3 days ago,3-5 years of directly related experience is re...
3,Business Analyst,LA Fitness,"Irving, TX 75039 (Freeport/Hackberry area)",EmployerActive Today,Exceptional analytical and problem-solving ski...
4,"Analyst, Revenue Operations Reporting",Equinix,"Frisco, TX 75034",PostedJust posted,Strong analytical skills with the ability to c...
...,...,...,...,...,...
224,Investor Reporting Analyst,Shellpoint Mortgage Servicing,"Coppell, TX",PostedPosted 5 days ago,Heavy use of Microsoft excel for data analysis...
225,Senior Lead Data Management Analyst - Data Def...,Wells Fargo,"Hybrid work in Irving, TX",PostedPosted 18 days ago,"Subject matter expertise on data governance, d..."
226,Pricing/Profit Improvement Analyst,TriMark USA,"Lewisville, TX 75067",PostedPosted 30+ days ago,3 – 5 years related experience.\n Analyst will...
227,ERP Business Analyst,HF Sinclair,"Dallas, TX 75201 (Uptown area)",PostedPosted 20 days ago,Advanced analytical and problem-solving skills...


In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/myueta/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [11]:
from collections import Counter
from nltk.corpus import stopwords
import pandas as pd
import re

# Sample DataFrame (replace with your DataFrame)
# df = pd.read_excel('your_dataframe.xlsx')

# Sample stop words and additional words to delete
del_words = set(stopwords.words('english'))
del_words.update(["days","agomore","postedposted","posted", "ago","postedjust","postedmore","including"])

# Function to clean text and tokenize into words
def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    words = text.lower().split()  # Convert to lowercase and split into words
    return [word for word in words if word not in del_words and not word.isdigit()]  # Remove stop words, additional words, and numbers

# Clean and tokenize 'Requirement' column
cleaned_requirements = df['Requirement'].apply(clean_text)

# Flatten the list of lists into a single list of words
all_words = [word for sublist in cleaned_requirements for word in sublist]

# Count the frequency of each word
word_freq = Counter(all_words)

# Get the top 10 most frequent words
top_100_words = word_freq.most_common(100)

# Display the top 100 words
print("Top 100 most frequent words in 'Requirement' column:")
for word, freq in top_100_words:
    print(f"{word}: {freq}")


Top 100 most frequent words in 'Requirement' column:
data: 279
experience: 76
years: 70
business: 61
skills: 43
analysis: 37
analyze: 36
ability: 34
strong: 33
analyst: 29
related: 28
information: 28
analytics: 24
support: 22
attention: 22
detail: 22
analytical: 22
provide: 21
sets: 19
complex: 18
working: 18
accuracy: 16
tx: 16
role: 15
work: 15
large: 15
job: 15
process: 15
employeractive: 12
required: 12
bachelors: 12
degree: 12
management: 12
field: 12
tools: 12
commercial: 12
technology: 12
determine: 12
requirements: 12
manner: 12
reports: 12
knowledge: 12
location: 12
summary: 12
prepare: 12
identify: 12
modeling: 10
drive: 10
query: 10
various: 9
tasks: 9
exceptional: 9
candidate: 9
excellent: 9
domain: 9
industrial: 9
needs: 9
translate: 9
actionable: 9
insights: 9
utilize: 9
present: 9
models: 9
solutions: 9
product: 9
problem: 9
andor: 9
thinking: 9
requires: 9
team: 9
problems: 9
position: 9
company: 9
entertainment: 9
quality: 9
minimum: 9
trends: 9
use: 9
tm: 9
problemsol